In [1]:
#It works for all the csv's at the same time, although there are some csv's with inaccurate extraction, it will simply output an extra csv containing the paths of those files.
#What it does?: It solves header repeatation problem by combining them. Header named with 'municipality' or 'upazilla' or 'thana/upazilla' or 'years' repeats in 2nd row, 3rd row, 4th(sometimes).

In [2]:
import os
import pandas as pd

In [3]:
#Definition=> this combines the headers and replaces the target-row header;
#Parameters=> target-row: is the main header,row_tocheck: is the number of sub-headers need to be combined
#Return=> first argument returns the number of header rows that has been checked, it'll be usefull to drop the redundant rows. Second one returns true if there is any csv without datas, otherewise returns false.
def combineHeaders(target_row, row_tocheck, data, check_filewithnodata = 0):
    target_headers = list()
    target_headers = data.iloc[[target_row]]
    row_checked = list();
    row_ttl, col_ttl = data.shape;
    for i in range(row_tocheck):
        next_row = target_row + i + 1;
        
        #Checks if outofbounderror occurs, some files doesn't have any datas so facing outofbounderror, if you want to track those files just add a new parameter filepath and print(filepath) in the if statement
        if(next_row > row_ttl - 1):
            check_filewithnodata = 1;
            return row_checked,check_filewithnodata;
        
        for col in range(col_ttl):
            #FirstIf=> To check wheather the next column is header row or data row, if data row than no need to go further
            if((col==1) and (target_headers.iloc[0,col]!=data.iloc[next_row,col])):
                #print('Data row:',next_row)
                return row_checked,check_filewithnodata;
            if ((col!=0) and (target_headers.iloc[0,col]!=data.iloc[next_row,col])): # ignore column 0
                
                # update
                data.iloc[target_row,col] = str(data.iloc[target_row,col]) + ' ' + str(data.iloc[next_row,col])
        #print('Header Rows: ',next_row)
        row_checked = row_checked + [next_row]
    return row_checked, check_filewithnodata;

In [5]:
originpath = "cleaned_CSVs/"
destpath = "cleaned_CSVs_headerscombined/"
districtList = os.listdir(originpath)#making a list of all the districts
filenameList_nodata = list();#list of files without datas
for districtname in districtList:
    filenameList = [filename for filename in os.listdir(originpath + districtname) if filename.endswith('.csv')]
    for filename in filenameList:
        filepath = originpath + districtname + "/" + filename;
    #         print(filepath);
        original_data = pd.read_csv(filepath)
        row_checked, check_filewithnodata = combineHeaders(2,2,original_data)
        
        #check csv's without any data
        if check_filewithnodata:
            filenameList_nodata = filenameList_nodata + [filepath]
            
        # drop redundant rows
        if row_checked:
            for row in row_checked:
                original_data.drop([row], axis=0, inplace=True)
        # drop one extra column
        original_data.drop(original_data.columns[0], axis=1, inplace=True)

        # reset the index after dropping rows
        original_data = original_data.reset_index(drop=True)

        filepath_parent = destpath + districtname
        os.makedirs(filepath_parent, exist_ok=True)
        filepath = filepath_parent + "/" + filename
        original_data.to_csv(filepath, index = True)

with open(destpath + "filenamelist_without_data.txt", 'w') as f:
    for filename in filenameList_nodata:
        f.write("%s\n" % filename)